In [1]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 35.5MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14492986984803633103
xla_global_id: -1
]


In [3]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization, Input, SimpleRNN
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [4]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [5]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [6]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [7, 8, 9, 10, 11, 12, 13]
RNN_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in RNN_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(RNN_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(SimpleRNN(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(SimpleRNN(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(SimpleRNN(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(SimpleRNN(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(SimpleRNN(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(SimpleRNN(units=c,return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'RNN', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'RNN_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 17640 ]
{
  "model": "RNN",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 21,
  "patience_list": 3,
  "MAE_normalize": 0.13309389033820682,
  "MSE_normalize": 0.01804598167621116,
  "RMSE_normalize": 0.13433533294041133,
  "MAPE_normalize": 0.1389803342357718,
  "MAE": 5107.613839285712,
  "MSE": 26576656.242623445,
  "RMSE": 5155.255206352392,
  "MAPE": 0.04968955576647853
}
Percent CPU Usage: 42.1
Percent Ram Usage: 9.9
2022-10-08 20:51:58.248369+07:00
PROGRESS: [ 2 / 17640 ]
{
  "model": "RNN",
  "AHB_number": "AHB_8",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 13,
  "patience_list": 3,
  "MAE_normalize": 0.07339615162262628,
  "MSE_normalize": 0.005514557418235686,
  "RMSE_normalize": 0.07426006610713248,
  "MAPE_normalize": 0.07663291117542703,
  "MAE": 2816.6495535714266,

{
  "model": "RNN",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 15,
  "patience_list": 3,
  "MAE_normalize": 0.47937712605843263,
  "MSE_normalize": 0.23114086632085004,
  "RMSE_normalize": 0.4807711163546018,
  "MAPE_normalize": 0.5014068879773996,
  "MAE": 18396.578124999996,
  "MSE": 340405220.45321214,
  "RMSE": 18450.073724872,
  "MAPE": 0.17907674709352822
}
Percent CPU Usage: 53.9
Percent Ram Usage: 10.3
2022-10-08 20:52:05.510247+07:00
PROGRESS: [ 6 / 17640 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 65536.73214285714,
  "MSE": 4312246743.484375,
  "RMSE": 65667.69939235252,
  "MAPE": 0.05356989854922349
}
Percent CPU Usage: 39.8
Percent Ram Usage: 11.2
2022-10-09 11:45:34.279465+07:00
PROGRESS: [ 17433 / 17640 ]
{
  "model": "RNN",
  "AHB_number": "Country_Level",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "RNN_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 45,
  "patience_list": 10,
  "MAE_normalize": 0.024895196700106377,
  "MSE_normalize": 0.0006295089690145709,
  "RMSE_normalize": 0.025090017317940833,
  "MAPE_normalize": 0.026029755684481105,
  "MAE": 16007.410714285714,
  "MSE": 260263237.78348213,
  "RMSE": 16132.676088717648,
  "MAPE": 0.013094000064017272
}
Percent CPU Usage: 63.2
Percent Ram Usage: 11.2
2022-10-09 11:45:36.633572+07:00
PROGRESS: [ 17434 / 17640 ]
{
  "model": "RNN",
  "AHB_number": "Country_Level",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "RNN_hidden_units": 8,


In [ ]:
name = "all_performance_RNN_5nsteps_AHB8-9-10-11-12-13-C"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_RNN_5nsteps_AHB8-9-10-11-12-13-C.json /content/drive/MyDrive/Depression
! cp /content/all_performance_RNN_5nsteps_AHB8-9-10-11-12-13-C.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,RNN_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,RNN,AHB_8,uni,1,8,0.2,2,5,21,3,0.133094,0.018046,0.134335,0.138980,5107.613839,2.657666e+07,5155.255206,0.049690
1,RNN,AHB_8,mul,1,8,0.2,2,5,13,3,0.073396,0.005515,0.074260,0.076633,2816.649554,8.121379e+06,2849.803285,0.027401
2,RNN,AHB_8,uni,1,8,0.2,4,5,14,3,0.332210,0.111367,0.333718,0.347285,12748.872768,1.640127e+08,12806.744657,0.124076
3,RNN,AHB_8,mul,1,8,0.2,4,5,21,3,0.183909,0.034865,0.186721,0.191920,7057.693080,5.134570e+07,7165.591736,0.068646
4,RNN,AHB_8,uni,1,8,0.2,8,5,15,3,0.479377,0.231141,0.480771,0.501407,18396.578125,3.404052e+08,18450.073725,0.179077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17635,RNN,Country_Level,mul,3,128,0.5,32,5,61,10,0.209876,0.044225,0.210298,0.219566,134948.339286,1.828439e+10,135219.773681,0.110419
17636,RNN,Country_Level,uni,3,128,0.5,64,5,15,10,0.480390,0.230796,0.480413,0.501819,308887.053571,9.542008e+10,308901.410464,0.252549
17637,RNN,Country_Level,mul,3,128,0.5,64,5,28,10,0.023888,0.000760,0.027567,0.025266,15359.964286,3.141776e+08,17725.057067,0.012637
17638,RNN,Country_Level,uni,3,128,0.5,128,5,40,10,0.505036,0.255065,0.505039,0.527412,324734.071429,1.054536e+11,324736.136484,0.265467
